In [1]:
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 處理找不到元素的工具
from selenium.common.exceptions import NoSuchElementException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 加入行為鍊 ActionChain (在 WebDriver 中模擬滑鼠移動、點繫、拖曳、按右鍵出現選單，以及鍵盤輸入文字、按下鍵盤上的按鈕等)
from selenium.webdriver.common.action_chains import ActionChains

# 加入鍵盤功能 (例如 Ctrl、Alt 等)
from selenium.webdriver.common.keys import Keys

# 強制等待 (執行期間休息一下)
import time
from time import sleep

# 匯入 regex 套件
import re

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import pprint

# 平行任務處理
from concurrent.futures import ThreadPoolExecutor as tpe

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")  #不開啟實體瀏覽器背景執行
# my_options.add_argument("--start-maximized")  #最大化視窗
my_options.add_argument("--incognito")  #開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)

In [2]:
# 開啟目標網頁
driver.get('https://e-service.cwb.gov.tw/HistoryDataQuery')
ac = ActionChains(driver)

# 選縣市(台北市)
# 沒有市場不需要爬的縣市:南投縣、新竹縣、嘉義縣、金門、連江
driver.get('https://e-service.cwb.gov.tw/HistoryDataQuery')
sleep(2)
county = Select(driver.find_element(By.CSS_SELECTOR,'select#stationCounty'))
county.select_by_index(0)

# 選測站:台北市只要4~15，其他廢棄
for a in range(5, 15):
    station = Select(driver.find_element(By.CSS_SELECTOR,'select#station'))
    station.select_by_index(a)

    # 輸入日期
    inputElement = driver.find_element(By.CSS_SELECTOR,'input#datepicker')
    inputElement.send_keys(f'2014-01-01')

    # 按查詢
    search = driver.find_element(By.CSS_SELECTOR, "img#doquery")
    search.click()
    ac.perform()

    # 切換視窗
    window_before = driver.window_handles[0]
    window_after = driver.window_handles[1]
    driver.switch_to.window(window_after)
    sleep(2)

    #迴圈下載
    i = 0
    while (i<=3245): 
        i = i + 1
        # 下載
        download = driver.find_element(By.CSS_SELECTOR, 'a#downloadCSV > input[type="image"]')
        download.click()
        sleep(1)
    
        handles = driver.window_handles
        handle2 = handles[-1]
        driver.switch_to.window(handle2)
        # 點擊下一天
        nextDay = driver.find_element(By.CSS_SELECTOR,'a#nexItem > input[type="image"]')
        nextDay.click()
        sleep(1.5)
    
    # 跳回原頁面
    driver.switch_to.window(window_before)
    sleep(2)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x006EACD3+2075859]
	Ordinal0 [0x0067EE61+1633889]
	Ordinal0 [0x0057B680+571008]
	Ordinal0 [0x005ABBD6+768982]
	Ordinal0 [0x005A1C76+728182]
	Ordinal0 [0x005C731C+881436]
	Ordinal0 [0x005A15BF+726463]
	Ordinal0 [0x005C7534+881972]
	Ordinal0 [0x005DB56A+963946]
	Ordinal0 [0x005C7136+880950]
	Ordinal0 [0x0059FEFD+720637]
	Ordinal0 [0x005A0F3F+724799]
	GetHandleVerifier [0x0099EED2+2769538]
	GetHandleVerifier [0x00990D95+2711877]
	GetHandleVerifier [0x0077A03A+521194]
	GetHandleVerifier [0x00778DA0+516432]
	Ordinal0 [0x0068682C+1665068]
	Ordinal0 [0x0068B128+1683752]
	Ordinal0 [0x0068B215+1683989]
	Ordinal0 [0x00696484+1729668]
	BaseThreadInitThunk [0x7626FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77837BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77837B8E+238]
